# Trait vs Trait
In this notebook we compare two distinct sets of traits (e.g, blood traits vs metabolic traits) in order to determine whether a machine learning classfier can learn to distinguish between the two sets, provided an aggregated dataset. The example in this notebook will utilize files containing blood and metabolic traits. 

In [1]:
import warnings
warnings.filterwarnings('ignore')
import csv
import pandas as pd
import dask.dataframe as dd
import sys
import os
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV, train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from matplotlib import pyplot as plt
plt.style.use('ggplot')

sys.path.insert(1, os.path.join(sys.path[0], '..'))
from Evaluator import Evaluator

/home/ec2-user/anaconda3/envs/chainer_p36/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


### Utility Functions

In [2]:
# For storage of the smaller dataframe
def write_to_S3(df, filepath):
    bytes_to_write = df.to_csv(None).encode()
    fs = s3fs.S3FileSystem(key=access_key, secret=secret_key)
    print (filepath + " written") 
    with fs.open(outpath + filepath, 'wb') as f:
        f.write(bytes_to_write)

In [3]:
# Return the intersection of two lists 
def intersect(a, b):
    return list(set(a) & set(b))

### Trait 1 

In [13]:
# Read in data for trait 1 into a dataframe 
filepath = "s3://voightlab-data/allgrouped/allgrouped_train.csv"
trait1_df = pd.read_csv(filepath, index_col=0)
trait1_df.head()

,MCF-7_ChIP-seq_CTCF_ENCSR000AHD_ENCFF001UML_ENCFF001UMN_intersect.bed,MCF-7_ChIP-seq_TAF1_ENCSR000AHF_ENCFF001UNU_ENCFF001UNT_intersect.bed,GM12878_ChIP-seq_CTCF_ENCFF002CDP.bed,K562_ChIP-seq_CTCF_ENCFF002CEL.bed,K562_ChIP-seq_POLR2A_ENCFF002CET.bed,endothelial_cell_of_umbilical_vein_ChIP-seq_CTCF_ENCFF002CEH.bed,endothelial_cell_of_umbilical_vein_ChIP-seq_POLR2A_ENCFF002CEJ.bed,keratinocyte_ChIP-seq_CTCF_ENCFF002CFA.bed,keratinocyte_ChIP-seq_POLR2A_ENCFF002CFC.bed,H1-hESC_ChIP-seq_H3K27me3_ENCFF001SUY.bed,...,Thyroid,Uterus,Vagina,Whole_Blood,phastcon,snpcount,is_BMI,is_CAD,is_T2D,is_lipids
9387,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1.000,0.004693,0,0,1,0
1208,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1.000,0.007920,0,0,0,0
4181,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0.001,0.000880,0,0,0,0
2804,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1.000,0.007334,0,0,0,1
2375,0,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0.912,0.027574,0,0,0,0


In [14]:
# Encode positive label in one column
labels = ['is_BMI', 'is_CAD', 'is_T2D', 'is_lipids']
trait1_df['label_1'] = 0

for label in labels:
    trait1_df['label_1'] = trait1_df['label_1'] + trait1_df[label]

# If any of the Trait 1 type labels are 1, this will encode as a positive instance
trait1_df['label_1'] = trait1_df['label_1'].apply(lambda x: 1 if x > 0 else 0)

# Get rid of the original labels now that we have mapped to a new one. 
trait1_df = trait1_df.drop(labels, axis=1)

### Trait 2 

In [15]:
# Replace with path to trait 2 
filepath = "s3://voightlab-data/allgrouped/"
trait2_df = pd.read_csv(filepath + "allgrouped_train.csv", index_col=0)
trait2_df.head()

,MCF-7_ChIP-seq_CTCF_ENCSR000AHD_ENCFF001UML_ENCFF001UMN_intersect.bed,MCF-7_ChIP-seq_TAF1_ENCSR000AHF_ENCFF001UNU_ENCFF001UNT_intersect.bed,GM12878_ChIP-seq_CTCF_ENCFF002CDP.bed,K562_ChIP-seq_CTCF_ENCFF002CEL.bed,K562_ChIP-seq_POLR2A_ENCFF002CET.bed,endothelial_cell_of_umbilical_vein_ChIP-seq_CTCF_ENCFF002CEH.bed,endothelial_cell_of_umbilical_vein_ChIP-seq_POLR2A_ENCFF002CEJ.bed,keratinocyte_ChIP-seq_CTCF_ENCFF002CFA.bed,keratinocyte_ChIP-seq_POLR2A_ENCFF002CFC.bed,H1-hESC_ChIP-seq_H3K27me3_ENCFF001SUY.bed,...,Thyroid,Uterus,Vagina,Whole_Blood,phastcon,snpcount,is_BMI,is_CAD,is_T2D,is_lipids
9387,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1.000,0.004693,0,0,1,0
1208,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1.000,0.007920,0,0,0,0
4181,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0.001,0.000880,0,0,0,0
2804,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1.000,0.007334,0,0,0,1
2375,0,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0.912,0.027574,0,0,0,0


In [16]:
# Encode positive label in one column
labels = ['is_BMI', 'is_CAD', 'is_T2D', 'is_lipids']
trait2_df['label_2'] = 0

for label in labels:
    trait2_df['label_2'] = trait2_df['label_2'] + trait2_df[label]

# If any of the Trait 1 type labels are 1, this will encode as a positive instance
trait2_df['label_2'] = trait2_df['label_2'].apply(lambda x: 1 if x > 0 else 0)

# Get rid of the original labels now that we have mapped to a new one
trait2_df = trait2_df.drop(labels, axis=1)

### Combining datasets
In order to train a machine learning model on both traits we need to ensure we only train on features that both datasets have in common. To do this we'll take the intersect of the two sets of feature names. Additionally, we'll have to choose how to encode the labels so that all of trait 1's positives and negatives are distinct from all of trait 2's positives and negatives. The best way to do this is to use a multiclass system, corresponding to the below:

* 0: Trait 1 negatives
* 1: Trait 1 positives
* 2: Trait 2 negatives
* 3: Trait 2 positives

In [17]:
# Encode labels, trait 1's labels are already in the format that we want them to be
trait1_df['label'] = trait1_df['label_1']
trait2_df['label'] = trait2_df['label_2'].apply(lambda x: 2 if x == 0 else 3)

# Now drop the original trait columns
trait1_df = trait1_df.drop(['label_1'], axis=1)
trait2_df = trait2_df.drop(['label_2'], axis=1)

In [18]:
common_feats = intersect(trait1_df.columns, trait2_df.columns)

trait1_cols = trait1_df[common_feats]
trait2_cols = trait2_df[common_feats]

# Combine into a single dataframe
df = pd.concat([trait1_cols, trait2_cols])

In [19]:
# Create training and test sets
# Select 25% of dataframe for a validation set
df = df.sample(frac=1)
idx = round(len(df) * .75)

# Split into two dataframes
train_df = df.iloc[:idx, :]
test_df = df.iloc[idx:, :]

In [20]:
# Create training and test splits, dropping labels and sample weights from training set
labels = ['label']

X_train = train_df.drop(labels, axis=1)
y_train = train_df.loc[:, labels]

X_test = test_df.drop(labels, axis=1)
y_test = test_df.loc[:, labels]

In [21]:
y_train.head()

,label
8733,2
2212,1
4518,1
5189,0
6717,2


### Model Training and Evaluation
Now that we've created a combined dataset, we're ready to train a multiclass classification model on it. 

In [22]:
ev = Evaluator()

#### Logistic Regression

In [ ]:
log_model = LogisticRegressionCV(solver='liblinear', 
                                 penalty='l1', 
                                 class_weight='balanced',
                                 scoring='roc_auc', 
                                 cv=10,
                                 max_iter=5000,
                                 n_jobs=-1)

log_model.fit(X_train, y_train)

In [ ]:
print("Training Dataset Logistic", end=" ")
ev.summarize_performance(log_model, X_train, y_train, threshold=0.5)

In [ ]:
print("Test Dataset Logistic", end=" ")
log_metrics = ev.summarize_performance(log_model, X_test, y_test, threshold=0.5, return_stats=True)

In [ ]:
ev.plot_mcroc_curve(log_model, X_test, y_test, classes=[0,1,2,3])

In [ ]:
# Plot feature importance
coefs = log_model.coef_
feature_names = X_train.columns
important_features = ev.feat_importance(coefs, feature_names, 20, one_dim=False)[0]

#### Random Forest

In [ ]:
# n_jobs = -1 allows training to be done on all available cores
rf_model = RandomForestClassifier(n_estimators=100,
                                  max_depth=5,
                                  n_jobs=-1, 
                                  class_weight='balanced', 
                                  oob_score=True)

rf_model.fit(X_train, y_train)

In [ ]:
ev.summarize_performance(rf_model, X_train, y_train, proba=True)

In [ ]:
ev.summarize_performance(rf_model, X_test, y_test, proba=True)

In [ ]:
ev.plot_mcroc_curve(rf_model, X_test, y_test, classes=[0,1,2,3])

In [ ]:
# Plot feature importance
coefs = rf_model.feature_importances_
feature_names = X_train.columns
important_features = ev.feat_importance(coefs, feature_names, 20, one_dim=True)